In [2]:
#!/usr/bin/env python3
import os, glob, numpy as np, mne, pandas as pd
from joblib import Parallel, delayed

In [14]:
# Configuration
BASE_DIR = '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/sleep-edf-database-expanded-1.0.0'
SUBFOLDERS = ['sleep-cassette', 'sleep-telemetry']
OUTPUT_DIR = '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/processed_sleepedf'
os.makedirs(OUTPUT_DIR, exist_ok=True)

USE_MULTIPLE_CHANNELS = True
if USE_MULTIPLE_CHANNELS:
    CHANNELS_TO_LOAD = ["EEG Fpz-Cz", "EOG horizontal"]
else:
    CHANNELS_TO_LOAD = ["EEG Fpz-Cz"]

TARGET_SFREQ = 100.0
LOW_FREQ = 0.5
HIGH_FREQ = 30.0
EPOCH_LENGTH = 30.0
SEQ_LENGTH = 20
SEQ_STRIDE = 10

ANNOTATION_MAP = {
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}

def process_record(psg_path, hyp_path, channels, target_sfreq, low_freq, high_freq, epoch_length):
    raw = mne.io.read_raw_edf(psg_path, include=channels, preload=True, verbose=False)
    if raw.info['sfreq'] != target_sfreq:
        raw.resample(target_sfreq, npad="auto", verbose=False)
    picks = mne.pick_types(raw.info, eeg=True, eog=True)
    raw.filter(l_freq=low_freq, h_freq=high_freq, picks=picks, verbose=False)
    ann = mne.read_annotations(hyp_path)
    raw.set_annotations(ann, emit_warning=False)
    events, _ = mne.events_from_annotations(raw, event_id=ANNOTATION_MAP, chunk_duration=epoch_length)
    tmin = 0.0; tmax = epoch_length - 1/raw.info['sfreq']
    epochs = mne.Epochs(raw, events=events, event_id=ANNOTATION_MAP, tmin=tmin, tmax=tmax,
                        baseline=None, preload=True, verbose=False)
    data = epochs.get_data()
    labels = epochs.events[:, -1]
    for ch in range(data.shape[1]):
        m = np.mean(data[:, ch, :])
        s = np.std(data[:, ch, :]) if np.std(data[:, ch, :]) != 0 else 1.0
        data[:, ch, :] = (data[:, ch, :] - m) / s
    return data, labels, raw.ch_names

def create_sequences(data, labels, seq_length, seq_stride):
    n_epochs = data.shape[0]
    sequences, seq_labels = [], []
    for start in range(0, n_epochs - seq_length + 1, seq_stride):
        sequences.append(data[start:start+seq_length])
        seq_labels.append(labels[start:start+seq_length])
    return np.array(sequences), np.array(seq_labels)

def find_hypnogram(psg_file):
    # Extract subject ID (assumes first 6 characters, e.g., "SC4001")
    subject_id = os.path.basename(psg_file)[:6]
    dir_path = os.path.dirname(psg_file)
    pattern = os.path.join(dir_path, f"{subject_id}*Hypnogram.edf")
    hyp_files = glob.glob(pattern)
    if len(hyp_files) == 1:
        return hyp_files[0]
    elif len(hyp_files) > 1:
        return hyp_files[0]  # or choose based on additional rules if needed
    else:
        return None

def process_and_save(psg_file, output_dir, channels):
    hyp_file = find_hypnogram(psg_file)
    if not hyp_file:
        print(f"Hypnogram not found for {psg_file}, skipping.")
        return
    try:
        data, labels, ch_names = process_record(psg_file, hyp_file, channels,
                                                  TARGET_SFREQ, LOW_FREQ, HIGH_FREQ, EPOCH_LENGTH)
    except Exception as e:
        print(f"Error processing {psg_file}: {e}")
        return
    rec_id = os.path.basename(psg_file).replace('-PSG.edf', '')
    np.savez_compressed(os.path.join(output_dir, f"{rec_id}_epochs.npz"),
                        data=data.astype('float32'), labels=labels.astype('int8'))
    sequences, seq_labels = create_sequences(data, labels, SEQ_LENGTH, SEQ_STRIDE)
    np.savez_compressed(os.path.join(output_dir, f"{rec_id}_sequences.npz"),
                        sequences=sequences.astype('float32'), seq_labels=seq_labels.astype('int8'))
    print(f"Processed {rec_id}: epochs {data.shape[0]}, sequences {sequences.shape[0]}, channels: {ch_names}")

In [15]:
def main():
    psg_files = []
    for sub in SUBFOLDERS:
        psg_files.extend(glob.glob(os.path.join(BASE_DIR, sub, '*-PSG.edf')))
    print(f"Found {len(psg_files)} PSG files.")
    Parallel(n_jobs=2)(delayed(process_and_save)(f, OUTPUT_DIR, CHANNELS_TO_LOAD) for f in psg_files)

if __name__ == '__main__':
    main()

Found 197 PSG files.
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]


/var/folders/g6/qyrrm6017k55qbd8_wywqxgw0000gn/T/ipykernel_26652/2945860575.py:41: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning


Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]


/var/folders/g6/qyrrm6017k55qbd8_wywqxgw0000gn/T/ipykernel_26652/2945860575.py:41: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning


Processed SC4201E0: epochs 2803, sequences 279, channels: ['EEG Fpz-Cz', 'EOG horizontal']
Processed SC4171E0: epochs 2741, sequences 273, channels: ['EEG Fpz-Cz', 'EOG horizontal']
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]


...

Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]


/var/folders/g6/qyrrm6017k55qbd8_wywqxgw0000gn/T/ipykernel_26652/2945860575.py:41: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning


Processed ST7052J0: epochs 1034, sequences 102, channels: ['EEG Fpz-Cz', 'EOG horizontal']
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]


/var/folders/g6/qyrrm6017k55qbd8_wywqxgw0000gn/T/ipykernel_26652/2945860575.py:41: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning


Processed ST7121J0: epochs 1026, sequences 101, channels: ['EEG Fpz-Cz', 'EOG horizontal']
Processed ST7192J0: epochs 931, sequences 92, channels: ['EEG Fpz-Cz', 'EOG horizontal']


In [6]:
OUTPUT_DIR = '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/processed_sleepedf'
rec_id = 'ST7121J0'  # Change this to any recording ID you want to inspect

# Load epoch-level data
epoch_file = os.path.join(OUTPUT_DIR, f"{rec_id}_epochs.npz")
epoch_data = np.load(epoch_file)
data = epoch_data['data']      # shape: (n_epochs, n_channels, n_samples)
labels = epoch_data['labels']  # shape: (n_epochs,)

print(f"Epoch-level data shape: {data.shape}")
print(f"Epoch-level labels shape: {labels.shape}")
print("First 10 labels:", labels[:100])

# Load sequence-level data
seq_file = os.path.join(OUTPUT_DIR, f"{rec_id}_sequences.npz")
seq_data = np.load(seq_file)
sequences = seq_data['sequences']       # shape: (n_sequences, SEQ_LENGTH, n_channels, n_samples)
seq_labels = seq_data['seq_labels']     # shape: (n_sequences, SEQ_LENGTH)

print(f"Sequence-level data shape: {sequences.shape}")
print(f"Sequence-level labels shape: {seq_labels.shape}")
print("First sequence labels:", seq_labels[0])


Epoch-level data shape: (1026, 2, 3000)
Epoch-level labels shape: (1026,)
First 10 labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 1 1 1 1 1 1 1 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1]
Sequence-level data shape: (101, 20, 2, 3000)
Sequence-level labels shape: (101, 20)
First sequence labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [5]:
hyp_path = '/Users/tereza/spring_2025/STAT_4830/STAT-4830-GOALZ-project/data/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4121EC-Hypnogram.edf'
ann = mne.read_annotations(hyp_path)
print("Annotation descriptions:", ann.description)

Annotation descriptions: ['Sleep stage W' 'Sleep stage 1' 'Sleep stage 2' 'Sleep stage 1'
 'Sleep stage 2' 'Sleep stage 3' 'Sleep stage 4' 'Sleep stage 3'
 'Sleep stage 4' 'Sleep stage 3' 'Sleep stage 4' 'Sleep stage 3'
 'Sleep stage 2' 'Sleep stage R' 'Sleep stage 1' 'Sleep stage 2'
 'Sleep stage 1' 'Sleep stage 2' 'Sleep stage 3' 'Sleep stage 2'
 'Sleep stage 3' 'Sleep stage 4' 'Sleep stage 3' 'Sleep stage 2'
 'Sleep stage 1' 'Sleep stage 2' 'Sleep stage W' 'Sleep stage 1'
 'Sleep stage 2' 'Sleep stage W' 'Sleep stage 1' 'Sleep stage 2'
 'Sleep stage W' 'Sleep stage 1' 'Sleep stage 2' 'Sleep stage R'
 'Sleep stage 2' 'Sleep stage 3' 'Sleep stage 2' 'Sleep stage 3'
 'Sleep stage 2' 'Sleep stage 3' 'Sleep stage 2' 'Sleep stage 3'
 'Sleep stage 2' 'Sleep stage 1' 'Sleep stage 2' 'Sleep stage 3'
 'Sleep stage 2' 'Sleep stage 3' 'Sleep stage 2' 'Sleep stage 3'
 'Sleep stage 4' 'Sleep stage 3' 'Sleep stage 4' 'Sleep stage 3'
 'Sleep stage 4' 'Sleep stage 3' 'Sleep stage 4' 'Sleep stage 3'
